In [53]:
#!pip install tomotopy

In [54]:
import tomotopy as tp
import pickle
import time as timer
import pathlib
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
def mallet_corpus_to_df(corpusFile: pathlib.Path):
    """Converts a Mallet corpus file (i.e., file required for the Mallet import command) to a pandas DataFrame

    Parameters
    ----------
    corpusFile: pathlib.Path
        Path to the Mallet corpus file

    Returns
    -------
    :   pandas.DataFrame
        DataFrame with the corpus
    """

    corpus = [line.rsplit(' 0 ')[1].strip() for line in open(
        corpusFile, encoding="utf-8").readlines()]
    indexes = [line.rsplit(' 0 ')[0].strip() for line in open(
        corpusFile, encoding="utf-8").readlines()]
    corpus_dict = {
        'id': indexes,
        'text': corpus
    }
    return pd.DataFrame(corpus_dict)

In [17]:
path_corpus = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/models_preproc/iter_0/corpus.txt")
df = mallet_corpus_to_df(path_corpus)
df

,id,text
0,100016,methods processes embedded_systems embed criti...
1,115153,rapid point_care platforms infectious_diseases...
2,115632,recognise adverse drug reactions regulatory_ag...
3,115861,vaccine phase_ii sofia_ref main_objective exte...
4,116030,translational quantitative toxicology medicine...
...,...,...
65571,190119289,early_detection skin_cancer endure detect skin...
65572,190134697,water_vapor turbo compression thermal cool maj...
65573,190151860,geotechnical genetic inverse poor track_record...
65574,190161902,artificial_intelligence musical preserve prese...


In [20]:
model_path = "/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/models_preproc/iter_0"
vocab_w2id = {}
with (pathlib.Path(model_path)/'vocabulary.txt').open('r', encoding='utf8') as fin:
    for i, line in enumerate(fin):
        wd = line.strip()
        vocab_w2id[wd] = i

vectorizer = CountVectorizer(vocabulary=vocab_w2id.keys(), tokenizer=lambda x: x.split())
bow = vectorizer.fit_transform(df.text.values.tolist())
bow = bow.toarray()

/home/lbartolome/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
df_lemas = df[["text"]].values.tolist()

In [9]:
tw = tp.TermWeight.ONE # term weighting scheme in TermWeight
min_cf = 0             # minimum collection frequency of words.
min_df = 0             # minimum document frequency of words.
rm_top = 0             # the number of top words to be removed. 
depth = 4              # the maximum depth level of hierarchy between 2 ~ 32767
alpha = 10.0           # hyperparameter of Dirichlet distribution for document-depth level
eta = 0.1              # hyperparameter of Dirichlet distribution for topic-word
gamma = 1.0            # concentration coeficient of Dirichlet Process
seed = None            # random seed
mycorpus = df_lemas  
transform = None       # a callable object to manipulate arbitrary keyword arguments for a specific topic model

In [ ]:
mdl = tp.HLDAModel(tw = tp.TermWeight.ONE , min_cf= 0, min_df= 0, rm_top = 0, depth = 2, alpha = 10.0, eta = 0.1, gamma = 1.0)

print("LLEGA 1")
for texts in mycorpus:
    mdl.add_doc(texts[0].split())
print("LLEGA 2")
mdl.train(0)
print("LLEGA 3")
print('Num docs:{}, Num Vocabs:{}, Total Words:{}'.format(
    len(mdl.docs), len(mdl.used_vocabs), mdl.num_words))

# Let's train the model
for i in range(0, 1000, 20):
    print('Iteration: {:04}, LL per word: {:.4}'.format(i, mdl.ll_per_word))
    mdl.train(20)
print('Iteration: {:04}, LL per word: {:.4}'.format(1000, mdl.ll_per_word))

mdl.summary()


LLEGA 1
LLEGA 2
LLEGA 3
Num docs:65576, Num Vocabs:20374, Total Words:4497518
Iteration: 0000, LL per word: -9.169
Iteration: 0020, LL per word: -8.102
Iteration: 0040, LL per word: -8.055


In [ ]:
for k in range(mdl.k):
    if not mdl.is_live_topic(k): continue
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=15))

In [ ]:
#save lda model for reuse
hlda_save = 'hlda.bin'
mdl.save(hlda_save)

In [ ]:
#load and print
#mdl = tp.HLDAModel.load(hlda_save) 
#for k in range(mdl.k):
#    if not mdl.is_live_topic(k): continue
#    print('Top 10 words of topic #{}'.format(k))
#    print(mdl.get_topic_words(k, top_n=10))

In [119]:
betas_ = np.load('/export/usuarios_ml4ds/lbartolome/Datasets/CORDIS/htm_variability_models/htm_6_tpcs_20230922/TMmodel/betas.npy')

In [123]:
betas_

array([[8.38666805e-05, 4.13074700e-05, 1.00000000e-12, ...,
        1.87761233e-05, 3.00417966e-05, 1.00000000e-12],
       [9.97873293e-05, 2.49468331e-05, 7.10984724e-05, ...,
        1.00000000e-12, 1.00000000e-12, 1.00000000e-12],
       [1.14584453e-04, 4.86983932e-05, 1.00000000e-12, ...,
        1.00000000e-12, 1.00000000e-12, 3.43753367e-05],
       [1.00000000e-12, 1.00000000e-12, 1.00000000e-12, ...,
        1.00000000e-12, 1.00000000e-12, 1.00000000e-12],
       [3.41553395e-05, 1.30115576e-04, 1.00000000e-12, ...,
        1.00000000e-12, 1.00000000e-12, 1.00000000e-12],
       [3.71241193e-05, 1.00000000e-12, 1.00000000e-12, ...,
        1.00000000e-12, 1.00000000e-12, 1.00000000e-12]])

In [55]:
mdl = tp.HDPModel.load("/export/usuarios_ml4ds/lbartolome/Repos/my_repos/UserInLoopHTM/data/models/hdp/cancer/run1.bin") 

In [36]:
len(mdl.vocabs)

20374

In [56]:
mdl.k

28

In [38]:
np.array([mdl.get_topic_word_dist(el) for el in range(mdl.k)]).shape

(96, 20374)

In [39]:
betas = []
for el in range(mdl.k):
    betas_i = mdl.get_topic_word_dist(el)
    betas.append(betas_i)
betas = np.array(betas)
betas.shape

(96, 20374)

In [40]:
betas_ds = np.copy(betas)
if np.min(betas_ds) < 1e-12:
    betas_ds += 1e-12
deno = np.reshape((sum(np.log(betas_ds)) / mdl.k), (len(mdl.vocabs), 1))
deno = np.ones((mdl.k, 1)).dot(deno.T)
betas_ds = betas_ds * (np.log(betas_ds) - deno)
betas_ds

array([[ 8.81692984e-04,  2.59725790e-02,  4.59628272e-02, ...,
        -5.19945132e-07, -5.15196468e-07, -5.21960272e-07],
       [-4.89315327e-05, -6.56981603e-05, -5.23421798e-05, ...,
         6.82128010e-05,  7.03226895e-05,  6.73174500e-05],
       [-4.89315327e-05, -6.56981603e-05, -5.23421798e-05, ...,
         6.82128010e-05,  7.03226895e-05,  6.73174500e-05],
       ...,
       [-4.89315327e-05, -6.56981603e-05, -5.23421798e-05, ...,
         6.82128010e-05,  7.03226895e-05,  6.73174500e-05],
       [-4.89315327e-05, -6.56981603e-05, -5.23421798e-05, ...,
         6.82128010e-05,  7.03226895e-05,  6.73174500e-05],
       [-4.89315327e-05, -6.56981603e-05, -5.23421798e-05, ...,
         6.82128010e-05,  7.03226895e-05,  6.73174500e-05]])

In [41]:
vocab_w2id = {}
vocab_id2w = {}
for i, wd in enumerate(mdl.vocabs):
    vocab_w2id[wd] = i
    vocab_id2w[str(i)] = wd

In [42]:
n_words = 15
tpc_descs = []
for i in range(mdl.k):
    words = [mdl.vocabs[idx2] for idx2 in np.argsort(betas_ds[i])[::-1][0:n_words]]
    tpc_descs.append((i, ', '.join(words)))


In [57]:
for i in range(mdl.k):
    print(f"{str(i)}: {mdl.get_topic_words(i)}")

0: [('p', 0.0131168682128191), ('resection', 0.010004081763327122), ('radiotherapy', 0.009040066972374916), ('follow', 0.008683649823069572), ('metastasis', 0.008545225486159325), ('liver', 0.007927361875772476), ('recurrence', 0.007178408093750477), ('association', 0.006843074690550566), ('complication', 0.006355567369610071), ('perform', 0.006251612212508917)]
1: [('diagnosis', 0.014050409197807312), ('rare', 0.008448166772723198), ('metastasis', 0.00786814745515585), ('mass', 0.005724803544580936), ('reveal', 0.005365085322409868), ('association', 0.005278328899294138), ('cutaneous', 0.005275866016745567), ('lung', 0.005232677329331636), ('cystic', 0.005156717728823423), ('finding', 0.004942099563777447)]
2: [('breast', 0.009248296730220318), ('pain', 0.008121589198708534), ('intervention', 0.00781586766242981), ('woman', 0.0076530976220965385), ('symptom', 0.006865671370178461), ('association', 0.006745707709342241), ('quality_life', 0.006426509469747543), ('survivor', 0.0058496803

In [44]:
tpc_descs = [el for el in tpc_descs if el[0] not in [0,1,2,3,4,5,6,85,89,92,93,94,95]]
tpc_descs 

[(7,
  'microtubule_mt, supersymmetric_gauge, place_september, supernova_explosion, astrophysical_phenomenon, atlantic_meridional, diagnostic_audit, coach_client, astrophysical_object, nucleosomes, arab_spring, calcium_image, resect, science_cafã, historiographic'),
 (8,
  'cellular, genetic, protein, cancer, deoxyribonucleic_acid, disease, mechanism, molecule, ribonucleic_acid, genomic, signal, control, pathway, tissue, novel'),
 (9,
  'quantum, molecule, electric, optic, spin, control, light, atom, magnetic, novel, interaction, electron, energy, photonic, surface'),
 (10,
  'galaxy, particle, energy, universe, observation, star, evolution, mass, dark_matter, search, detector, measurement, solar, formation, large_hadron_collider'),
 (11,
  'energy, battery, fuel, cellular, electric, efficiency, carbon_dioxide, emission, hydrogen, vehicle, component, solar, plant, heat, reduction'),
 (12,
  'patient, cancer, therapeutic, treatment, cellular, clinical, disease, drug, tumor, diagnosis, b

In [45]:
n_words = 15
tpc_descs = []
for i in range(mdl.k):
    words = [mdl.vocabs[idx2] for idx2 in np.argsort(betas_ds[i])[::-1][0:n_words]]
    tpc_descs.append((i, words))
    
root_desc = tpc_descs[0][1]
tpc_descs = [el[1] for el in tpc_descs if el[0] !=0]
PC_pair_groups = [[root_desc, el] for el in tpc_descs]

In [46]:
from collections import Counter, defaultdict

def compute_CLNPMI(parent_diff_words, child_diff_words, all_bow, vocab):
    npmi_list = list()

    for p_w in parent_diff_words:
        flag_n = all_bow[:, vocab[p_w]] > 0
        p_n = np.sum(flag_n) / len(all_bow)

        for c_w in child_diff_words:
            flag_l = all_bow[:, vocab[c_w]] > 0
            p_l = np.sum(flag_l)
            p_nl = np.sum(flag_n * flag_l)

            if p_nl == len(all_bow):
                npmi_score = 1
            else:
                p_l = p_l / len(all_bow)
                p_nl = p_nl / len(all_bow)
                p_nl += 1e-10
                npmi_score = np.log(p_nl / (p_l * p_n)) / -np.log(p_nl)

            npmi_list.append(npmi_score)

    return npmi_list


def get_CLNPMI(PC_pair_groups, all_bow, vocab):
    CNPMI_list = list()
    for group in tqdm(PC_pair_groups):
        layer_CNPMI = []
    
       
        parent_words = set(group[0])
        child_words = set(group[1])

        inter = parent_words.intersection(child_words)
        parent_diff_words = list(parent_words.difference(inter))
        child_diff_words = list(child_words.difference(inter))
        
        npmi_list = compute_CLNPMI(parent_diff_words, child_diff_words, all_bow, vocab)

        # Handle repetitive word pair NPMI assignments if necessary
        num_repetition = (
            len(parent_words) - len(parent_diff_words)
        ) * (len(child_words) - len(child_diff_words))
        npmi_list.extend([-1] * num_repetition)

        layer_CNPMI.extend(npmi_list)
    
        CNPMI_list.append(np.mean(layer_CNPMI))  # Append the group's result


    return CNPMI_list

def compute_diff_topic_pair(topic_words_a, topic_words_b):
    word_counter = Counter()
    word_counter.update(topic_words_a)
    word_counter.update(topic_words_b)
    diff = (np.asarray(list(word_counter.values())) == 1).sum() / (len(topic_words_a) + len(topic_words_b))
    return diff


def get_topics_difference(topic_pair_groups):
    diff_list = list()
    for group in tqdm(topic_pair_groups):
        layer_diff = list()
        diff = compute_diff_topic_pair(group[0], group[1])
        layer_diff.append(diff)
        diff_list.append(np.mean(layer_diff))

    return diff_list

In [47]:
CLNPMI = get_CLNPMI(PC_pair_groups, bow, vocab_w2id)

100%|██████████| 95/95 [00:25<00:00,  3.77it/s]


In [48]:
np.mean(CLNPMI).round(3)

-0.095

In [49]:
PC_TD = get_topics_difference(PC_pair_groups)

100%|██████████| 95/95 [00:00<00:00, 50784.97it/s]


In [50]:
np.mean(PC_TD).round(3)

0.971